# Defining the aoi

In [ ]:
import pandas as pd
import numpy as np
import xarray as xa
from pathlib import Path
import src.data.ecostress_io as eio
import rioxarray
import sys
import geopandas as gpd
import json
import dask
from dask.distributed import Client
import matplotlib.pyplot as plt

n_partitions = 8
root_path = Path("/mnt/ecostress/rhone-ecostress-data/")

with open(f"{root_path}/geo-countries/archive/countries.geojson", "rb") as f:
    all_countries_geojson = json.loads(f.read())


for i in all_countries_geojson['features']:
    if i['properties']['ADMIN'] == "France":
        france_geo = i


france_gdf = gpd.GeoDataFrame.from_features([france_geo]).explode()

france_gdf[france_gdf.area==max(france_gdf.area)].plot()

aoi = france_gdf[france_gdf.area==max(france_gdf.area)]

aoi_geojson = json.loads(aoi.to_json())

xmin, ymin, xmax, ymax = aoi.total_bounds
bounds_tuple = (4, 42, 7, 47)
xmin, ymin, xmax, ymax = bounds_tuple  # hardcoding since concattenating 1000s of ecostress files with different overlaps hangs

# Global rivers dataset

In [ ]:
rivers_df = gpd.read_file(Path(root_path, "europe_rivers/eu_river.shp"))

rivers_df['R_ID'] = rivers_df['R_ID'].apply(int).apply(str)

france_rivers_df = rivers_df.cx[xmin:xmax, ymin:ymax]

aoi.crs = france_rivers_df.crs # setting crs for aoi

### Check to make sure they overlay, geopandas has poor support for linestrings so we subset by the bounding box

In [ ]:
base = aoi.plot(color="grey", edgecolor="black")

france_rivers_df.plot(ax=base, color="blue")

## reading in paths and example data array

In [ ]:
qa_path = Path(root_path, "ECO3ANCQA")
et_path = Path(root_path, "ECO3ETPTJPL")
esi_path = Path(root_path, "ECO4ESIPTJPL")

whole_tif_qa_paths, csv_qa_paths, xml_qa_paths = eio.separate_extensions(qa_path)

whole_tif_etdaily_paths, csv_et_paths, xml_et_paths = eio.separate_extensions(et_path, "*ETdaily*.tif")

Clipping all et tifs, takes about 10 min with threading. 

In [ ]:
tempdir = Path("/home/ryan/work/tmp")

clipped_scene_paths = [Path(p) for p in tempdir.glob("*clipped*")]

if clipped_scene_paths == []:
    client = Client()
    
    batches = eio.batches_from(whole_tif_etdaily_paths, 8)
    
    batch_results = []
    
    for batch in batches:
    
        batch_result = dask.delayed(eio.clip_and_save)(batch, bounds_tuple, filter_nan = True, outDir=tempdir)
        batch_results.append(batch_result)
        
    result_futures = client.compute(batch_results, scheduler='processes')

    clipped_scene_paths = [i.result() for i in result_futures if i.result()]# gets rid of None that denotes too little scene overlap

    client.restart()
    

In [ ]:
france_river_lines = france_rivers_df.copy()
buffered_france_rivers_df = france_rivers_df.to_crs(epsg=2154)\
                                            .buffer(5000)\
                                            .to_crs(epsg=4326) # buffers by 5000 meters
france_rivers_df['geometry'] = buffered_france_rivers_df

In [ ]:
resolution = eio.read_ecostress_scene(clipped_scene_paths[0]).rio.resolution()
crs = france_rivers_df.crs
aoi_grid = eio.gdf_to_dataarray(france_rivers_df, crs, resolution)

In [ ]:
resampled_scene_paths = [Path(p) for p in tempdir.glob("*resampled*")]

if resampled_scene_paths == []:
    client = Client()
    batches = eio.batches_from(resampled_scene_paths, n_partitions)

    def wrapper(paths, aoi_grid, tempdir, path_id):
        return_paths = []
        for path in paths:
            x = eio.read_mask_ecostress_scene(path)
            y = eio.resample_xarray_to_basis(x, aoi_grid)
            return_paths.append(eio.write_tmp(y, tempdir, path_id))
        return return_paths

    all_results = []
    for batch in batches:
        sub_result = dask.delayed(wrapper)(batch, aoi_grid, tempdir, "resampled")
        all_results.append(sub_result)
        
    result_future = client.compute(all_results, scheduler="processes")
    
    resampled_scene_paths = [i.result() for i in result_future]
    
    client.restart()

In [ ]:
resampled_data_arrays = eio.read_scenes(resampled_scene_paths)

In [ ]:
et_tseries = xa.concat(resampled_data_arrays, dim="date").sortby('date')

In [ ]:
from geocube.api.core import make_geocube
france_rivers_df['value'] = 1 # allow sus to make non empty dataset, required for resampling
river_arr = make_geocube(vector_data=france_rivers_df, resolution=resolution)['value']

In [ ]:
reanalysis_path = Path(root_path, "era5-download.nc")

In [ ]:
met_dataset = xa.open_dataset(reanalysis_path, chunks = {"time": 1, "latitude": 39, "longitude": 41})

In [ ]:
met_dataset['vpd'] = eio.vapor_deficit(met_dataset['t2m']-273.15,met_dataset['d2m']-273.15)

In [ ]:
met_dataset = met_dataset.rio.set_crs(4326)

In [ ]:
vpd = met_dataset['vpd']

In [ ]:
vpd = vpd.rio.set_crs(4326)

Need to reduce hourly vpd before reprojecting because too much memory

local time in utc is 1 hour behind paris, so I take the mean of vpd over 10am-3pm local time

In [ ]:
daytime_mask = np.isin(vpd.time.dt.hour, [9, 10, 11, 12, 13, 14])

In [ ]:
daytime_vpd = vpd.isel(time=daytime_mask)

In [ ]:
daytime_vpd_daily = daytime_vpd.resample(time="1D").mean()

In [ ]:
daytime_vpd_daily

I need this much mem to reproject...

In [ ]:
np.float32(1).itemsize * np.prod([395, 6101, 6558]) / 1e9

In [ ]:
from rasterio.enums import Resampling
daytime_vpd_daily.rio.reproject_match(et_tseries[0], resampling = Resampling.bilinear)

Plotting ecostress availability

In [ ]:
et_tseries = et_tseries.sel(band=1)

et_2018_may_sept=et_tseries.sel(date=slice("2018-06-01", "2018-09-30"))

et_2018_may_sept=et_2018_may_sept.chunk(chunks={"date": 101, "y": 1000, "x": 1000})

et_2018_may_sept = et_2018_may_sept.where(et_2018_may_sept > 0)

et_nonnan_count = ~np.isnan(et_2018_may_sept)

true_count = et_nonnan_count.astype(bool).sum(dim="date")

true_count_c = true_count.compute()

f, ax = plt.subplots(1)
true_count_c.plot.imshow(ax=ax)
plt.title("Number of ECOSTRESS Observations, May-Sept 2018")
france_river_lines.plot(ax=ax, color="red")

Aggregating hourly data to dekad

In [ ]:
pev_dekad = met_dataset['pev'].resample(time='1D').sum().resample(time='10D').mean()

pev_dekad_2018 = pev_dekad.sel(time=slice("2018-01-01", "2018-10-01"))

met_dataset['pev'].sel(time="2019-04-01")

pev_dekad_2019.plot(x='longitude', y='latitude', col='time', col_wrap=5)

## pretty ET plotting

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
%matplotlib inline

ETcolors = ["#f6e8c3", "#d8b365", "#99974a", "#53792d", "#6bdfd2", "#1839c5"]
ETcmap = LinearSegmentedColormap.from_list("ET", ETcolors)
date_utc = pd.to_datetime(et['date'].values)
layer_type = et.attrs['filename'].split("_")[-3]
title = 'ECO3ETPTJPL Evapotranspiration'

fig = plt.figure(figsize=(9.7,7.6))                                                       # Set the figure size (x,y)
fig.suptitle(f'{title} ({layer_type}) \n at {date_utc}', fontsize=22)  # Add title for the plots
plt.axis('off')                                                                           # Remove axes from plot
im = plt.imshow(et.sel(band=1), cmap=ETcmap);                                                        # Plot array using colormap
# plt.scatter(Tcol, Trow, color="black", marker='x')                                        # Plot tower location
# Add a colormap legend
plt.colorbar(im, orientation='horizontal', fraction=0.05, pad=0.004, label=f"ET ({et.attrs['units']})", shrink=0.6).outline.set_visible(True)

## Code graveyard

trying to plot xarray image data and geopandas data with ipyleaflet but it can't do image overlays yet (unless it comes from a server via url potentially)

In [ ]:
import ipyleaflet as ipyl
x = france_river_lines.unary_union.envelope.centroid.xy[0][0]
y = france_river_lines.unary_union.envelope.centroid.xy[1][0]
e, n, w, s =true_count_c.rio.bounds()
m = ipyl.Map(center = (y,x), zoom=6)

rivers_data = ipyl.GeoData(geo_dataframe = france_river_lines,
                   style={'color': 'purple', 'opacity':3, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Rivers')
m.add_layer(rivers_data)
plt.imsave("observation_count.jpeg",true_count_c)
obs_heatmap = ipyl.ImageOverlay(
    url="observation_count.jpeg",
    bounds=((s, w), (n, e))
)

m.add_layer(obs_heatmap)
m

trying to plot ecostress DataArray with geopandas shapes

In [ ]:
def cartopy_project_geo_df(df, crs):

    # This can be converted into a `proj4` string/dict compatible with GeoPandas
    crs_proj4 = crs.proj4_init
    return df.to_crs(crs_proj4)

crs = ccrs.PlateCarree()
aoi_projected = cartopy_project_geo_df(aoi, crs)
france_rivers_df_projected = cartopy_project_geo_df(france_rivers_df, crs)
# base = aoi_projected.plot(color="grey", edgecolor="black")
# france_rivers_df_projected.plot(ax=base, color="blue")


import cartopy.crs as ccrs
import matplotlib.pyplot as plt
ax = plt.axes(projection=crs)
all_et_daily[14].sel(band=1).plot.imshow(ax=ax, transform=crs)
# ax.add_geometries(aoi_projected['geometry'], crs=crs)
ax.add_geometries(france_rivers_df_projected['geometry'], crs=crs)

In [ ]:
def mask_NA_values():
    """
    Daily ET products have both nan values from where there are clouds 
    and -1e+13 for where the ecostress swath was clipped during the ordering process
    """
    masked_et = np.ma.masked_where(et.sel(band=1) == np.nan, et.sel(band=1))
    masked_et = np.ma.masked_where(masked_et == -1e+13, masked_et)